In [5]:
# --- IMPORTS ---
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt
import pandas as pd
import datetime

# --- STRATEGY LOGIC ---
class OpeningRangeBreakout(bt.Strategy):
    params = (
        ('opening_range_minutes', 5),
        ('relative_volume_lookback', 14),
        ('atr_period', 14),
        ('max_positions', 20),
        ('stop_loss_risk_size', 0.01), # 1% of allocated capital
    )

    def __init__(self):
        """
        Initialization method. This is called once for the strategy.
        """
        self.orders = {}
        self.stocks_in_play = []
        self.daily_data = {}

        # Create indicators and data structures for each stock in the universe
        for d in self.datas:
            self.daily_data[d._name] = {}
            self.daily_data[d._name]['atr'] = bt.indicators.AverageTrueRange(d, period=self.p.atr_period)

            # Store daily volume for relative volume calculation
            d.vol_history = bt.ind.SMA(d.volume, period=self.p.relative_volume_lookback)

            # Timer to trigger selection logic
            self.add_timer(
                when=bt.Timer.SESSION_START,
                offset=datetime.timedelta(minutes=self.p.opening_range_minutes),
                weekdays=[0, 1, 2, 3, 4] # Monday to Friday
            )

        self.portfolio_value_start_day = self.broker.get_value()

    def log(self, txt, dt=None):
        """
        Logging function for this strategy.
        """
        dt = dt or self.datas.datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        """
        Handles order notifications.
        """
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f"BUY EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}")
            elif order.issell():
                self.log(f"SELL EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}")

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f"Order Canceled/Margin/Rejected: {order.info.get('name', 'N/A')}")

        # Clear the order reference
        if order.ref in self.orders:
            del self.orders[order.ref]

    def notify_timer(self, timer, when, *args, **kwargs):
        """
        Triggered by the timer set in __init__. This is where we select our "stocks in play".
        """
        if self.datas.datetime.time() == datetime.time(9, 35):
            self.select_stocks_in_play()

    def select_stocks_in_play(self):
        """
        Calculates relative volume and selects the top stocks.
        """
        relative_volumes = []
        for d in self.datas:
            try:
                # Get volume of the first 5 minutes
                current_5min_volume = d.volume[0]
                # Get average 5-minute volume from daily data (simplified for this example)
                # A more accurate implementation would store historical 5-min volumes
                avg_5min_volume = d.vol_history[-1] / (6.5 * 60 / self.p.opening_range_minutes)

                if avg_5min_volume > 0:
                    rel_vol = current_5min_volume / avg_5min_volume
                    if rel_vol > 1 and self.daily_data[d._name]['atr'][0] > 0.5 and d.close[0] > 5:
                        relative_volumes.append((d._name, rel_vol))
            except IndexError:
                continue

        # Sort by relative volume and take the top N
        relative_volumes.sort(key=lambda x: x[1], reverse=True)
        self.stocks_in_play = [item[0] for item in relative_volumes[:self.p.max_positions]]
        self.log(f"Stocks in Play: {self.stocks_in_play}")

    def next(self):
        """
        Main strategy logic, called on each bar.
        """
        # Liquidate all positions at the end of the day
        if self.datas.datetime.time() >= datetime.time(15, 55):
            for d in self.datas:
                if self.getposition(d).size != 0:
                    self.close(d, exectype=bt.Order.Market)
            return

        # Only trade stocks selected as "in play"
        for d in self.datas:
            if d._name not in self.stocks_in_play:
                continue

            # Check if we are already in the market for this stock
            if self.getposition(d).size != 0:
                continue

            # Check if we have an open order for this stock
            if d in self.orders:
                return

            # Define the opening range (assuming 5-minute bars)
            # Find the high and low of the first 5 minutes of the day
            opening_range_high = max(d.high.get(size=self.p.opening_range_minutes))
            opening_range_low = min(d.low.get(size=self.p.opening_range_minutes))


            # Entry Logic
            # Check if the current bar's close is above the opening range high (for long)
            if d.close[0] > opening_range_high:
                entry_price = opening_range_high
                stop_price = entry_price - (2 * self.daily_data[d._name]['atr'][0])

                # Position Sizing
                allocated_capital = self.portfolio_value_start_day / self.p.max_positions
                risk_per_share = entry_price - stop_price
                if risk_per_share > 0:
                    size = (self.p.stop_loss_risk_size * allocated_capital) / risk_per_share

                    # Concentration Cap
                    max_size = self.broker.get_cash() / entry_price / self.p.max_positions
                    size = min(size, max_size)

                    # Place stop-market order to go long
                    order = self.buy(data=d, size=size, exectype=bt.Order.Stop, price=entry_price, valid=self.datas.datetime.date(0) + datetime.timedelta(days=1))
                    order.addinfo(name='Long Entry')
                    self.orders[d] = order

            # Check if the current bar's close is below the opening range low (for short)
            elif d.close[0] < opening_range_low:
                entry_price = opening_range_low
                stop_price = entry_price + (2 * self.daily_data[d._name]['atr'][0])

                # Position Sizing
                allocated_capital = self.portfolio_value_start_day / self.p.max_positions
                risk_per_share = stop_price - entry_price
                if risk_per_share > 0:
                    size = (self.p.stop_loss_risk_size * allocated_capital) / risk_per_share

                    # Concentration Cap
                    max_size = self.broker.get_cash() / entry_price / self.p.max_positions
                    size = min(size, max_size)

                    # Place stop-market order to go short
                    order = self.sell(data=d, size=size, exectype=bt.Order.Stop, price=entry_price, valid=self.datas.datetime.date(0) + datetime.timedelta(days=1))
                    order.addinfo(name='Short Entry')
                    self.orders[d] = order

    def start(self):
        self.portfolio_value_start_day = self.broker.get_value()

    def prenext(self):
        # Update portfolio value at the start of each day for position sizing
        if self.datas.datetime.date(0) != self.datas.datetime.date(-1):
            self.portfolio_value_start_day = self.broker.get_value()


# --- BACKTEST EXECUTION ---
if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # --- Add Data Feeds ---
    # In a real scenario, you would loop through your universe of stocks and add their data feeds
    # For this example, we'll use a few sample data files.
    # Replace with your own data paths and symbols.
    data_files = {
        'AAPL': 'path/to/your/AAPL-5min.csv',
        'MSFT': 'path/to/your/MSFT-5min.csv',
        'GOOG': 'path/to/your/GOOG-5min.csv',
        #... add all 1000 stocks from your universe here
    }

    for ticker, filepath in data_files.items():
        data = bt.feeds.GenericCSVData(
            dataname=filepath,
            dtformat=('%Y-%m-%d %H:%M:%S'),
            datetime=0,
            open=1,
            high=2,
            low=3,
            close=4,
            volume=5,
            openinterest=-1,
            timeframe=bt.TimeFrame.Minutes,
            compression=5,
            name=ticker
        )
        cerebro.adddata(data)

    # Add SPY as a benchmark
    spy_data = bt.feeds.YahooFinanceCSVData(dataname='path/to/your/SPY-daily.csv')
    cerebro.adddata(spy_data)

    # Add the strategy
    cerebro.addstrategy(OpeningRangeBreakout)

    # Set initial capital
    cerebro.broker.set_cash(100000.0)

    # Add Analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

    # --- Run the backtest ---
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # --- Print Analysis ---
    strat = results[0]
    print('\n--- PERFORMANCE METRICS ---')
    print(f"Sharpe Ratio: {strat.analyzers.sharpe.get_analysis()['sharperatio']:.2f}")
    print(f"Max Drawdown: {strat.analyzers.drawdown.get_analysis().max.drawdown:.2f}%")
    print(f"Annualized Return: {strat.analyzers.returns.get_analysis()['rnorm100']:.2f}%")

    trade_analysis = strat.analyzers.trades.get_analysis()
    if trade_analysis.total.total > 0:
        print(f"Total Trades: {trade_analysis.total.total}")
        print(f"Win Rate: {(trade_analysis.won.total / trade_analysis.total.total) * 100:.2f}%")
        print(f"Average Win: {trade_analysis.won.pnl.average:.2f}")
        print(f"Average Loss: {trade_analysis.lost.pnl.average:.2f}")

    # --- Plot the results ---
    cerebro.plot(style='candlestick')

Starting Portfolio Value: 100000.00


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/AAPL-5min.csv'

In [3]:
!pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 9.1 MB/s eta 0:00:00
